**Gives access to Google Drive env**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Unzip Final_Dataset + loads the content on Google Collab local temporary storage = faster image reading during training**

In [ ]:
!unzip /content/drive/MyDrive/Final_Dataset.zip -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/Final_Dataset/train/notepad/177_jpg.rf.2b2b6c154010008b55e0d5e97ccb7033.jpg  
  inflating: /content/Final_Dataset/train/notepad/179_jpg.rf.739415c59cc8501fce9fce7765d71fcf.jpg  
  inflating: /content/Final_Dataset/train/notepad/17_jpg.rf.4a2a8f897aa3755514909902bde6722b.jpg  
  inflating: /content/Final_Dataset/train/notepad/17_jpg.rf.6b0631bb5370fbc9bd6940188a295e43.jpg  
  inflating: /content/Final_Dataset/train/notepad/17_jpg.rf.7eaa6b0f5b58f7c96b22f480fa2aa78e.jpg  
  inflating: /content/Final_Dataset/train/notepad/17_jpg.rf.9d79ac4ad45aef0085d57b199a62d892.jpg  
  inflating: /content/Final_Dataset/train/notepad/17_jpg.rf.e8d69036db493f12420f5ba9b7734c56.jpg  
  inflating: /content/Final_Dataset/train/notepad/180_jpg.rf.224a09e4db20ec8add31dffcca76d820.jpg  
  inflating: /content/Final_Dataset/train/notepad/180_jpg.rf.60dc726322d8d62a0a0c2890305c1dda.jpg  
  inflating: /content/Final_Dataset/train/notepad/181_

**Training and Saving the Office Item Image Classification Model (ResNet50 Transfer Learning)**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os
import time
from tqdm import tqdm

# --- Configuration ---
DATA_DIR = '/content/Final_Dataset'
MODEL_SAVE_PATH = '/content/drive/MyDrive/office_item_classifier.pth'

NUM_EPOCHS = 25
BATCH_SIZE = 32
LEARNING_RATE = 0.001

def train_model():
    print("--- Starting Model Training ---")

    # --- Step 1/5: Setting up data transformations ---
    print("\n[Step 1/5] Setting up data transformations...")
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'validation': transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    print("Transformations defined.")

    # --- Step 2/5: Loading datasets and creating dataloaders ---
    print("\n[Step 2/5] Loading datasets...")
    image_datasets = {x: datasets.ImageFolder(os.path.join(DATA_DIR, x), data_transforms[x])
                      for x in ['train', 'validation']}

    dataloaders = {x: DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
                   for x in ['train', 'validation']}

    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'validation']}
    class_names = image_datasets['train'].classes
    num_classes = len(class_names)
    print(f"Datasets loaded. Found {num_classes} classes: {', '.join(class_names)}")

    # --- Step 3/5: Handling Class Imbalance ---
    print("\n[Step 3/5] Calculating class weights for imbalance...")
    train_labels = [sample[1] for sample in image_datasets['train'].samples]
    class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
    class_weights = torch.tensor(class_weights, dtype=torch.float)
    print("Class weights calculated.")

    # --- Step 4/5: Setting up the Model, Optimizer, and Loss Function ---
    print("\n[Step 4/5] Initializing ResNet50 model for transfer learning...")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Training on device: {device.type.upper()}")

    class_weights = class_weights.to(device)

    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    for param in model.parameters():
        param.requires_grad = False

    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.fc.parameters(), lr=LEARNING_RATE)
    print("Model, optimizer, and loss function are ready.")

    # --- Step 5/5: Starting the Training Loop ---
    print("\n[Step 5/5] Starting model training... ")
    start_time = time.time()
    for epoch in range(NUM_EPOCHS):
        print(f'\nEpoch {epoch+1}/{NUM_EPOCHS}')
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Use tqdm for a clear progress bar on each epoch
            progress_bar = tqdm(dataloaders[phase], desc=f"{phase.capitalize()} Epoch {epoch+1}")
            for inputs, labels in progress_bar:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                # Update progress bar description
                progress_bar.set_postfix(loss=loss.item())

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    time_elapsed = time.time() - start_time
    print(f'\n--- Training Complete ---')
    print(f'Finished in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

    # --- Save the Final Model ---
    torch.save(model.state_dict(), MODEL_SAVE_PATH)
    print(f"\nModel saved successfully to your Google Drive at: {MODEL_SAVE_PATH}")

if __name__ == '__main__':
    train_model()

--- Starting Model Training ---

[Step 1/5] Setting up data transformations...
Transformations defined.

[Step 2/5] Loading datasets...
Datasets loaded. Found 10 classes: chair, desk lamp, headphones, keyboard, monitor, mouse, mug, notepad, pen, table

[Step 3/5] Calculating class weights for imbalance...
Class weights calculated.

[Step 4/5] Initializing ResNet50 model for transfer learning...
Training on device: CUDA
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 171MB/s]


Model, optimizer, and loss function are ready.

[Step 5/5] Starting model training... 

Epoch 1/25
----------


Train Epoch 1: 100%|██████████| 312/312 [01:14<00:00,  4.17it/s, loss=0.39]


Train Loss: 0.9634 Acc: 0.7520


...

Validation Loss: 0.1655 Acc: 0.9432

Epoch 25/25
----------


Train Epoch 25: 100%|██████████| 312/312 [01:13<00:00,  4.22it/s, loss=0.0742]


Train Loss: 0.3541 Acc: 0.8828


Validation Epoch 25: 100%|██████████| 67/67 [00:08<00:00,  8.37it/s, loss=0.0852]


Validation Loss: 0.1693 Acc: 0.9418

--- Training Complete ---
Finished in 34m 21s

Model saved successfully to your Google Drive at: /content/drive/MyDrive/office_item_classifier.pth
